# Script work for Generating Charting Outputs

In [1]:
#! pip install cpi

In [1]:
import numpy as np
import pandas as pd
from siuba import *

import altair as alt
import altair_saver
from plotnine import *

from IPython.display import Markdown

from shared_utils import altair_utils
from shared_utils import geography_utils
from shared_utils import calitp_color_palette as cp
from shared_utils import styleguide

from calitp import to_snakecase
import intake

import _clean_data
import _dla_utils

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
E0316 22:09:26.968163373    1109 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using e

In [2]:
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

In [7]:
df= pd.read_parquet("dla_df.parquet")

In [4]:
len(df)

21024

In [27]:
#subset= 11

In [28]:
#df = df>>filter(_.dist==subset)

In [15]:
def labeling(word):
    # Add specific use cases where it's not just first letter capitalized
    LABEL_DICT = { "prepared_y": "Year",
              "dist": "District",
              "total_requested": "Total Requested",
              "fed_requested":"Fed Requested",
              "ac_requested": "Advance Construction Requested",
              "nunique":"Number of Unique",
              "project_no": "Project Number"}
    
    if (word == "mpo") or (word == "rtpa"):
        word = word.upper()
    elif word in LABEL_DICT.keys():
        word = LABEL_DICT[word]
    else:
        word = word.replace('n_', 'Number of ').title()
        word = word.replace('unique_', "Number of Unique ").title()
        word = word.replace('_', ' ').title()
    
    return word


In [14]:
def basic_bar_chart_test(df, x_col, y_col, color_col, subset, chart_title=''):

    if chart_title == "":
        chart_title = (f"{labeling(x_col)} by {labeling(y_col)}")

    
    chart = (alt.Chart(df)
             .mark_bar()
             .encode(
                 x=alt.X(x_col, title=labeling(x_col), sort=('-y')),
                 y=alt.Y(y_col, title=labeling(y_col)),
                 color = alt.Color(color_col,
                                  scale=alt.Scale(
                                      range=altair_utils.CALITP_SEQUENTIAL_COLORS),
                                      legend=alt.Legend(title=(labeling(color_col)))
                                  ))
             .properties( 
                          title=chart_title)
    )

    chart=styleguide.preset_chart_config(chart)
   # chart.save(f"./chart_outputs/d{subset}_outputs/bar_{x_col}_by_{y_col}.png")
    
    return chart


def basic_line_chart_test(df, x_col, y_col, subset, chart_title=''):
    
    if chart_title == "":
        chart_title = (f"{labeling(x_col)} by {labeling(y_col)}")
    
    
    chart = (alt.Chart(df)
             .mark_line()
             .encode(
                 x=alt.X(x_col, title=labeling(x_col)),
                 y=alt.Y(y_col, title=labeling(y_col))
                                   )
              ).properties( 
                          title=chart_title)

    chart=styleguide.preset_chart_config(chart)
    #chart.save(f"./chart_outputs/d{subset}_outputs/line_{x_col}_by_{y_col}.png")
    
    return chart


In [33]:
df_years= _dla_utils.count_all_years(df)

In [34]:
basic_bar_chart_test(df_years, 'prepared_y', 'total_requested_sum', 'prepared_y', subset, 'Total Requested Funds by Year')


alt.Chart(...)

In [35]:
#df = (df>>filter(_.dist==subset))
df_top = _dla_utils.find_top(df)

basic_bar_chart_test((df_top>>filter(_.variable=='primary_agency_name')), 'value', 'count', 'count', subset, chart_title='Primary Agency name')

alt.Chart(...)

In [36]:
df_top = _dla_utils.find_top(df)

In [37]:
df_top>>filter(_.variable=='prefix')


,value,count,variable
0,HSIPL,3433,prefix
1,CML,3414,prefix
2,STPL,2878,prefix
3,ER,1786,prefix
4,BRLO,1482,prefix
5,BRLS,940,prefix
6,ATPL,620,prefix
7,RPSTPL,589,prefix
8,HPLUL,340,prefix
9,SRTSL,331,prefix


## Transit Agencies

In [38]:
transit_agencies = df[df['primary_agency_name'].str.contains(
              'Transit|tranist|Rail',
              case=False, na=False)]

In [39]:
transit_agencies>>count(_.primary_agency_name)

,primary_agency_name,n
0,Alameda - Contra Costa Transit District,7
1,Amador Rapid Transit System,2
2,Antelope Valley Transit Authority,1
3,Central Contra Costa Transit Authority,6
4,Fresno County Rural Transit Agency,1
5,Golden Empire Transit District,6
6,Kern Regional Transit,2
7,Kings County Area Public Transit Agency,2
8,Livermore Amador Valley Transit Authority,3
9,Marin County Transit District,1


In [40]:
#Add to chart function
chart_title='Obligations by Year'
chart_title.replace(" ", "_")

'Obligations_by_Year'

## Charts

In [42]:
def make_charts(df, subset):     
    df= pd.read_parquet("dla_df.parquet")
    
    df = (df>>filter(_.dist==subset))
    
    #adding a column
    df['process_days']= (df['dist_processing_days'] + df['hq_processing_days'] + df['fhwa_processing_days'])
    
    #subsetting the data
    df_years = _dla_utils.count_all_years(df)
    df_top = _dla_utils.find_top(df)
    
    transit = df[df['type_of_work'].str.contains(
              'Transit|bus|Bus|Rideshare',
              case=False, na=False)]
    
    transit_agencies = df[df['primary_agency_name'].str.contains(
              'Transit|Rail',
              case=False, na=False)]
    
    
    ##print statements
    display(Markdown(f"# **District {subset} Analysis**"))
    
    
    
    display(Markdown(f"**Unique Agencies Funding**"))
    display(Markdown(f'There are {(df.primary_agency_name.nunique())} Unique Agencies'))
    
    
#     display(Markdown(f"**Transit Funding**"))
#     display(Markdown(f"Out of {len(df)} obligations, {len(transit)} are transit related"))
    
#     display(Markdown(f"**Transit Agencies**"))
#     display(Markdown(f'There are {(len(transit_agencies>>count(_.primary_agency_name)))} Transit Agencies'))
    
    
    ##tables
    
    display(Markdown(f"**Number of Unique Prefix Codes by Agency**"))
    display((_dla_utils.get_nunique(df, 'prefix', 'primary_agency_name')).head(5))
    
    display(Markdown(f"**Number of Unique Agencies by Prefix Codes**"))
    display((_dla_utils.get_nunique(df, 'primary_agency_name', 'prefix')).head(5))
    
    display(Markdown(f"**Top 5 Types of Work**"))
    display((transit>>count(_.type_of_work)>>arrange(-_.n)>>select(_.type_of_work)).head(5))
    
    
#     ###charting df_years
    
#     column_names = list(df_years)

#     for column in column_names:
    
#         display(Markdown(f"**{labeling(column)} Over Prepared Year**"))
#         display(basic_bar_chart_test(df_years, "prepared_y", column, "prepared_y", subset))
    
    
#     ###charting df_tops 
    
#     values = sorted(df_top['variable'].unique())

#     for value in values:
#         display(Markdown(f"**Top Values in {labeling(value)}**"))
#         display(basic_bar_chart_test((df_top>>filter(_.variable==value)), 'value', 'count', 'count', subset,
#                                     chart_title=f'Top 20 {labeling(value)}'))
        
        
       
   #Other Charting
    
    #Line chart for Obligations by Year
    chart1= (basic_line_chart_test((df_top>>filter(_.variable=='prepared_y')), 'value', 'count', subset, 'Obligations by Year'))
    display(chart1)
    
    
    #Bar chart Agencies With The Most Obligations
    chart2= (basic_bar_chart_test((df_top>>filter(_.variable=='primary_agency_name')), 'value', 'count', 'value', subset, 'Agencies With The Most Obligations'))
    display(chart2)
    
    
    #Bar chart Agencies With The Most Unique Prefix Codes
    chart3 = (basic_bar_chart_test(((_dla_utils.get_nunique(df, 'prefix', 'primary_agency_name')).head(30)),
                            'primary_agency_name', 'n', 'primary_agency_name', subset, 'Agencies With The Most Unique Prefix Codes'))
    display(chart3)
    
    
    #Bar chart Average Total Requested Funds by Agency
    chart4=(basic_bar_chart_test((((_dla_utils.calculate_data_all(df, 'total_requested', 'primary_agency_name', aggfunc="mean"))
                          >>arrange(-_.total_requested)).head(30)
                        ), 'primary_agency_name','total_requested', 'primary_agency_name', subset,
                           'Average Total Requested Funds by Agency'
                       ))
    display(chart4)
    
    
    #Bar chart Bottom Average Total Requested Funds by Agency
    avg_funds_bottom = (df>>group_by(_.primary_agency_name)>>summarize(avg_funds=_.total_requested.mean())>>arrange(-_.avg_funds)).tail(50)

    chart5=( basic_bar_chart_test((avg_funds_bottom.tail(40)), 'primary_agency_name','avg_funds', 'primary_agency_name', subset, 
                          'Lowest Average Total Funds by Agency'))
    display(chart5)
   

    
    #Bar chart Average Total Requested Funds by Agency: Transit Related Funding
    chart6 = (basic_bar_chart_test((((_dla_utils.calculate_data_all(transit, 'total_requested', 'primary_agency_name', aggfunc="mean"))
                          >>arrange(-_.total_requested)).head(30)
                        ), 'primary_agency_name','total_requested', 'primary_agency_name', subset,
                           'Average Total Requested Funds by Agency: Transit Related Funding'
                       ))
    display(chart6)
    
    # #Bar chart Transit Agencies Average Funding
    # chart7 = ( basic_bar_chart_test((_dla_utils.calculate_data_all(transit_agencies, 'total_requested', aggregate_by='primary_agency_name', aggfunc="mean")),
    #                        'primary_agency_name', 'total_requested', 'primary_agency_name', subset,
    #                        'Transit Agencies Average Funding'))
    # display(chart7)
    
    # Bar chart Average Total Requested Funds by Prefix
    chart8 = (basic_bar_chart_test((((_dla_utils.calculate_data_all(df, 'total_requested', 'prefix', aggfunc="mean"))
                          >>arrange(-_.total_requested)).head(30)), 'prefix','total_requested', 'prefix', subset,
                           'Average Total Requested Funds by Prefix'
                       ))
    display(chart8)
    
    #work categories info and charts
    display(Markdown(f"**Work Categories**"))
    
    work_cat = ['active_transp', 'transit', 'bridge', 'street','freeway', 'infra_resiliency_er',
       'congestion_relief']
    
    for i in work_cat:
        subset = (df>>filter(_[i]==1))
        subset_2 = ((_dla_utils.find_top(subset))>>filter(_.variable=='primary_agency_name')
                >>select(_.value,_.count)).head(5)
        subset_2['Percent of Category'] = (((subset_2['count'])/(len(subset)))*100)
        subset_2 =subset_2.rename(columns = {'value':'Agency', 'count':f'{_dla_utils.labeling(i)} Obligations'})
    
    
        #generate chart:
    
        subset_3= ((subset.groupby(['primary_agency_name']).agg({i:'sum',
                                                    'process_days':'mean',
                                                    'adjusted_total_requested':'mean',
                                                    'adjusted_fed_requested':'mean',
                                                    'adjusted_ac_requested':'mean'}).reset_index())>>arrange(-_[i])).head(5)
    
        subset_3 =subset_3.rename(columns = {'primary_agency_name':'Agency',
                                         'adjusted_total_requested':'Total Requested',
                                         'adjusted_fed_requested':'Fed Requested',
                                         'adjusted_ac_requested':'AC Requested'})
    
        subset_4 = pd.melt(subset_3, id_vars=['Agency'],
            value_vars=['Total Requested','Fed Requested','AC Requested'],
            var_name='Categories', value_name='value'
           )

        ## following cell block makes it hard to change name
        # subset_4 = (subset_3 >> gather('category', 'value', _.adjusted_total_requested,
        #                 _.adjusted_fed_requested,
        #                 _.adjusted_ac_requested,
        #                 ))
    
        chart = (alt.Chart(subset_4).mark_bar().encode(
            x=alt.X('value', axis=alt.Axis(format='$', title='Obligated Funding ($2021)')),
            y=alt.Y("Agency"),
            color='Categories:N',
            row='Categories:N'
        ))
    
        display(Markdown(f'**Top Agencies using {_dla_utils.labeling(i)} Projects**'))
        display(subset_2.style.format(formatter={('Percent of Category'): "{:.2f}%"}))
        display(chart)
    
    

In [47]:
#parameters cell
subset = 4

In [48]:
make_charts(df, subset)

# **District 4 Analysis**

**Unique Agencies Funding**

There are 151 Unique Agencies

**Number of Unique Prefix Codes by Agency**

,primary_agency_name,n
28,Contra Costa County,18
107,San Francisco County,17
110,San Jose,17
119,Santa Clara County,17
2,Alameda County,16


...

,Agency,Infra Resiliency Er Obligations,Percent of Category
0,Contra Costa County,101,5.20%
1,Napa County,88,4.53%
2,Santa Clara County,81,4.17%
3,Oakland,70,3.60%
4,Sonoma County,69,3.55%


alt.Chart(...)

**Top Agencies using Congestionumber Of Relief Projects**

,Agency,Congestionumber Of Relief Obligations,Percent of Category
0,Contra Costa Transportation Authority,6,22.22%
1,Metropolitan Transportation Commission,5,18.52%
2,Sonoma County Transportation Authority,4,14.81%
3,San Mateo,3,11.11%
4,Solano Transportation Authority,3,11.11%


alt.Chart(...)